In [1]:
import json
import datetime
import pandas as pd
import collections

In [48]:
def strp(date_string):
    return datetime.datetime.strptime(date_string, '%Y-%m-%d %H:%M:%S.%f') 

def get_type(events, type, attribute_name, occurance=1):
    c_occurance = 0
    for e in events:
        type_object = e["type"]
        if type == type_object: 
            c_occurance += 1
            if c_occurance < occurance: continue 
            return e[attribute_name]
    return None

def get_pod_events_from_deployment_name(deployment_name, pod_names):
    for pod_name in pod_names:
        if deployment_name in pod_name: return pod_name
    return None 

In [46]:
def native_single_deployment_to_dataframe(filename):
    with open(filename, "r") as f:
        experiments = json.load(f)
    
    assert experiments[0]["single_deployment"] # Only one deployment multiple pods

    rows = []
    for e in experiments:
        data = e["data"]

        framework = "native"
        num_deployments = 1
        num_pods = e["pod_count"]
        num_iterations = e["num_iterations"]
        current_iteration = e["iteration"]
        api_call_datetime = data["time_before_create"]

        # Deployment specific
        deployment_name = get_type(data["deployment_events"], "created", "name")
        deployment_creation_date = get_type(data["deployment_events"], "created", "time")
        deployment_schedule_date = get_type(data["deployment_events"], "modified", "time", 2)
        deployment_deletion_date = get_type(data["deployment_events"], "deleted", "time")

        for pod_name in data["pod_events"]:
            pod_name = get_type(data["pod_events"][pod_name], "created", "name")
            pod_creation_date = get_type(data["pod_events"][pod_name], "created", "time")
            pod_schedule_date = get_type(data["pod_events"][pod_name], "modified", "time")
            pod_deletion_date = get_type(data["pod_events"][pod_name], "deleted", "time")

            row = {
                "framework": framework,
                "num_deployments": num_deployments,
                "num_pods": num_pods,
                "current_iteration": current_iteration,
                "num_iterations": num_iterations,
                "api_call_datetime": api_call_datetime,

                "deployment_name": deployment_name,
                "deployment_creation_date": deployment_creation_date,
                "deployment_schedule_date": deployment_schedule_date,
                "deployment_deletion_date": deployment_deletion_date,

                "pod_name": pod_name,
                "pod_creation_date": pod_creation_date,
                "pod_schedule_date": pod_schedule_date,
                "pod_deletion_date": pod_deletion_date,
            }
            rows.append(row)
    
    return pd.DataFrame(rows)

,framework,num_deployments,num_pods,current_iteration,num_iterations,api_call_datetime,deployment_name,deployment_creation_date,deployment_schedule_date,deployment_deletion_date,pod_name,pod_creation_date,pod_schedule_date,pod_deletion_date
0,native,1,1,0,50,2023-03-26 11:33:24.591891,deployment,2023-03-26 11:33:24.850995,2023-03-26 11:33:25.409881,2023-03-26 11:33:38.854166,deployment-3-665bcd7c99-96g8g,2023-03-26 11:33:24.633621,None,None
1,native,1,1,0,50,2023-03-26 11:33:24.591891,deployment,2023-03-26 11:33:24.850995,2023-03-26 11:33:25.409881,2023-03-26 11:33:38.854166,deployment-12-665bcd7c99-pct4p,2023-03-26 11:33:24.639341,None,None
2,native,1,1,0,50,2023-03-26 11:33:24.591891,deployment,2023-03-26 11:33:24.850995,2023-03-26 11:33:25.409881,2023-03-26 11:33:38.854166,deployment-0-665bcd7c99-m7mk8,2023-03-26 11:33:24.644722,2023-03-26 11:34:11.621108,2023-03-26 11:34:16.976779
3,native,1,1,0,50,2023-03-26 11:33:24.591891,deployment,2023-03-26 11:33:24.850995,2023-03-26 11:33:25.409881,2023-03-26 11:33:38.854166,deployment-1-665bcd7c99-gk4zl,2023-03-26 11:33:24.650026,None,None
4,native,1,1,0,50,2023-03-26 11:33:24.591891,deployment,2023-03-26 11:33:24.850995,2023-03-26 11:33:25.409881,2023-03-26 11:33:38.854166,deployment-19-665bcd7c99-m5g5r,2023-03-26 11:33:24.655534,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2720,native,1,20,49,50,2023-03-26 14:17:11.755309,deployment,2023-03-26 14:17:11.817159,2023-03-26 14:17:12.252321,2023-03-26 14:20:41.025755,deployment-665bcd7c99-25vh6,2023-03-26 14:17:12.914215,2023-03-26 14:17:13.298998,None
2721,native,1,20,49,50,2023-03-26 14:17:11.755309,deployment,2023-03-26 14:17:11.817159,2023-03-26 14:17:12.252321,2023-03-26 14:20:41.025755,deployment-665bcd7c99-c4872,2023-03-26 14:17:12.924125,2023-03-26 14:17:13.293803,None
2722,native,1,20,49,50,2023-03-26 14:17:11.755309,deployment,2023-03-26 14:17:11.817159,2023-03-26 14:17:12.252321,2023-03-26 14:20:41.025755,deployment-665bcd7c99-dcm64,2023-03-26 14:17:12.928940,2023-03-26 14:17:13.314116,None
2723,native,1,20,49,50,2023-03-26 14:17:11.755309,deployment,2023-03-26 14:17:11.817159,2023-03-26 14:17:12.252321,2023-03-26 14:20:41.025755,deployment-665bcd7c99-zxb9g,2023-03-26 14:17:12.933476,2023-03-26 14:17:13.303916,None


In [60]:
def native_multi_deployment_to_dataframe(filename):
    with open(filename, "r") as f:
        experiments = json.load(f)
    
    assert not experiments[0]["single_deployment"] # Only one deployment multiple pods

    rows = []
    for e in experiments:
        data = e["data"]

        framework = "native"
        num_deployments = e["pod_count"]
        num_pods = 1
        num_iterations = e["num_iterations"]
        current_iteration = e["iteration"]
        api_call_datetime = data["time_before_create"]

        # Deployment specific
        for deployment_name in data["deployment_events"]:

            deployment_name = get_type(data["deployment_events"][deployment_name], "created", "name")
            deployment_creation_date = get_type(data["deployment_events"][deployment_name], "created", "time")
            deployment_schedule_date = get_type(data["deployment_events"][deployment_name], "modified", "time", 2)
            deployment_deletion_date = get_type(data["deployment_events"][deployment_name], "deleted", "time")

            pod_name = get_pod_events_from_deployment_name(deployment_name, data["pod_events"].keys())

            # When we haven't received any event info from the pod so we simply skip
            if pod_name is None:
                continue

            pod_name = get_type(data["pod_events"][pod_name], "created", "name")
            pod_creation_date = get_type(data["pod_events"][pod_name], "created", "time")
            pod_schedule_date = get_type(data["pod_events"][pod_name], "modified", "time")
            pod_deletion_date = get_type(data["pod_events"][pod_name], "deleted", "time")

            row = {
                "framework": framework,
                "num_deployments": num_deployments,
                "num_pods": num_pods,
                "current_iteration": current_iteration,
                "num_iterations": num_iterations,
                "api_call_datetime": api_call_datetime,

                "deployment_name": deployment_name,
                "deployment_creation_date": deployment_creation_date,
                "deployment_schedule_date": deployment_schedule_date,
                "deployment_deletion_date": deployment_deletion_date,

                "pod_name": pod_name,
                "pod_creation_date": pod_creation_date,
                "pod_schedule_date": pod_schedule_date,
                "pod_deletion_date": pod_deletion_date,
            }
            rows.append(row)
    
    return pd.DataFrame(rows)

native_multi_deployment_to_dataframe("jsons/native1/20230326_053008.json")

,framework,num_deployments,num_pods,current_iteration,num_iterations,api_call_datetime,deployment_name,deployment_creation_date,deployment_schedule_date,deployment_deletion_date,pod_name,pod_creation_date,pod_schedule_date,pod_deletion_date
0,native,1,1,0,50,2023-03-26 05:30:08.172657,deployment-0,2023-03-26 05:30:08.457002,2023-03-26 05:30:09.174807,2023-03-26 05:30:20.746486,deployment-0-665bcd7c99-mz5hd,2023-03-26 05:30:08.684341,2023-03-26 05:30:09.165134,2023-03-26 05:30:54.877582
1,native,1,1,1,50,2023-03-26 05:30:54.907105,deployment-0,2023-03-26 05:30:55.003151,2023-03-26 05:30:55.196547,2023-03-26 05:31:04.985978,deployment-0-665bcd7c99-svztg,2023-03-26 05:30:55.146973,2023-03-26 05:30:55.165890,2023-03-26 05:31:39.080062
2,native,1,1,2,50,2023-03-26 05:31:39.109431,deployment-0,2023-03-26 05:31:39.176413,2023-03-26 05:31:39.376958,2023-03-26 05:31:49.211117,deployment-0-665bcd7c99-ppsz2,2023-03-26 05:31:39.327105,2023-03-26 05:31:39.344813,2023-03-26 05:32:24.379104
3,native,1,1,3,50,2023-03-26 05:32:24.414539,deployment-0,2023-03-26 05:32:24.529733,2023-03-26 05:32:24.728254,2023-03-26 05:32:34.467258,deployment-0-665bcd7c99-tm5tl,2023-03-26 05:32:24.650505,2023-03-26 05:32:24.731292,2023-03-26 05:33:08.563603
4,native,1,1,4,50,2023-03-26 05:33:08.598337,deployment-0,2023-03-26 05:33:08.651026,2023-03-26 05:33:08.838804,2023-03-26 05:33:17.872921,deployment-0-665bcd7c99-lpxlt,2023-03-26 05:33:08.776946,2023-03-26 05:33:08.832288,2023-03-26 05:33:52.105388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1271,native,20,1,49,50,2023-03-26 11:31:02.489650,deployment-15,2023-03-26 11:31:07.394735,2023-03-26 11:31:08.694865,None,deployment-15-665bcd7c99-tcwvb,2023-03-26 11:31:02.643020,2023-03-26 11:32:23.029213,None
1272,native,20,1,49,50,2023-03-26 11:31:02.489650,deployment-16,2023-03-26 11:31:07.625478,2023-03-26 11:31:08.658492,None,deployment-16-665bcd7c99-vgq68,2023-03-26 11:31:02.603485,None,None
1273,native,20,1,49,50,2023-03-26 11:31:02.489650,deployment-17,2023-03-26 11:31:07.883030,2023-03-26 11:31:08.913284,None,deployment-17-665bcd7c99-x24r8,2023-03-26 11:31:02.743272,None,None
1274,native,20,1,49,50,2023-03-26 11:31:02.489650,deployment-18,2023-03-26 11:31:08.243687,2023-03-26 11:31:09.408953,None,deployment-18-665bcd7c99-9zvts,2023-03-26 11:31:02.613861,None,None
